In [1]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm

path = '/nfs/misc/literature/shyama/OTAR_Old_Pipeline/AbsJan2020/'

In [2]:
all_files = sorted(glob.glob(path+'Annot_med*.xml'))

In [3]:
len(all_files)

10657

In [4]:
all_files[0]

'/nfs/misc/literature/shyama/OTAR_Old_Pipeline/AbsJan2020/Annot_medline.0.xml'

In [ ]:
def getfileblocks(file_path):

    subFileBlocks = []

    with open(file_path, 'r') as fh:
        for line in fh:
            if line.startswith('<!DOCTYPE'):
                subFileBlocks.append(line)
            else:
                subFileBlocks[-1] += line
                
    return subFileBlocks

In [5]:
def getfileblocks_abs(file_path):

    subFileBlocks = []

    with open(file_path, 'r') as fh:
        first_line = fh.readline()
        for line in fh:
            if line.startswith('<PubmedArticle>'):
                subFileBlocks.append(line)
            else:
                subFileBlocks[-1] += line
                
    return subFileBlocks



In [6]:
def GP_soup(file_soup):
    dict_sentences = defaultdict(list)
    for each_ztag in file_soup.find_all('z:uniprot'):
        try:
            gene_sentence = each_ztag.findParents('plain')[0].text
            dict_sentences[gene_sentence].append(each_ztag) 
        except:
            pass
    return dict_sentences

def GP_soup_abs(file_soup):
    dict_sentences = defaultdict(list)
    for each_ztag in file_soup.find_all('uniprot'):
        try:
            gene_sentence = each_ztag.findParents('plain')[0].text
            dict_sentences[gene_sentence].append(each_ztag) 
        except:
            pass
    return dict_sentences

from fuzzywuzzy import fuzz 
def compare_with_ml_annotations(r, z_tags):
    FP_removed_z_tags = []
    for each_z_tag in z_tags:
        for each_ml_annotation in r.json()['annotations']:
            score = fuzz.token_set_ratio(each_ml_annotation[3],each_z_tag.text)
            if  score ==100 and each_ml_annotation[2]=='GP':
                FP_removed_z_tags.append(each_z_tag)
    
    return list(set(z_tags) - set(FP_removed_z_tags))



In [7]:
import requests

def get_unique_fp_tags(GP_dict_sentences):
#     url = 'http://ai-capo-api-1:5200/spaCy_ner_predictor?text_sentence='
    url = 'http://ai-capo-api-lb/spaCy_ner_predictor?text_sentence='
    FP_dict_sentences =defaultdict(list)
    set_FPs = set()
    
    for each_uniprot_sentence, z_tags in GP_dict_sentences.items():
        r = requests.get(url+each_uniprot_sentence)
        if r.status_code == 200 and r.json()['status']==200:
            FPs = compare_with_ml_annotations(r, z_tags)
            if FPs:
                FP_dict_sentences[each_uniprot_sentence].append(FPs)
        else:
            print(each_uniprot_sentence)
        
    for FP_sent,FP_tag in FP_dict_sentences.items():
        for each_ztag in FP_tag[0]:
            set_FPs.add(each_ztag)

    return list(set_FPs)
        

In [ ]:
# path_new = '/nfs/misc/literature/Santosh_Tirunagari/OTAR_dumps/FP_removed/'
path_new = '/nfs/gns/literature/machine-learning/'
# for each_file_content in subFileBlocks:
for each_file_path in all_files:
    ss = getfileblocks(each_file_path)
    for each_article in tqdm(ss):
        soup = BeautifulSoup(each_article, 'html.parser')
        GP_dict_sentences = GP_soup(soup)
        if GP_dict_sentences:
            unique_fps = get_unique_fp_tags(GP_dict_sentences)

            for each_fp in unique_fps:
                if each_fp:
                    for tag in soup.findAll('z:uniprot', attrs={'ids':each_fp['ids']}):
                        tag.unwrap()

            new_file_content = str(soup).replace('\n<ebiroot','<ebiroot').replace('<sent','<SENT').replace('</sent>','</SENT>')

            with open(path_new+'edited_'+each_file_path.split('/')[-1], 'at') as f:
                f.write(new_file_content)



In [10]:
# path_new = '/nfs/misc/literature/Santosh_Tirunagari/OTAR_dumps/FP_removed/'
path_new = '/nfs/gns/literature/machine-learning/'
# for each_file_content in subFileBlocks:
for each_file_path in all_files:
    
    with open(path_new+'edited_'+each_file_path.split('/')[-1], 'a') as fl:
        fl.write('<MedlineCitationSet>\n')
    
    ss = getfileblocks_abs(each_file_path)
    for each_article in tqdm(ss):
        soup = BeautifulSoup(each_article, 'xml')
        GP_dict_sentences = GP_soup_abs(soup)
        if GP_dict_sentences:
            unique_fps = get_unique_fp_tags(GP_dict_sentences)

            for each_fp in unique_fps:
                if each_fp:
                    for tag in soup.findAll('uniprot', attrs={'ids':each_fp['ids']}):
                        tag.unwrap()

            new_file_content = str(soup).replace('<?xml version="1.0" encoding="utf-8"?>\n','').replace('<uniprot','<z:uniprot').replace('</uniprot','</z:uniprot').replace('<efo','<z:efo').replace('</efo','</z:efo')

            with open(path_new+'edited_'+each_file_path.split('/')[-1], 'a') as f:
                f.write(new_file_content)

    with open(path_new+'edited_'+each_file_path.split('/')[-1], 'a') as fe:
        fe.write('\n</MedlineCitationSet>\n')

In [ ]:
# offset = 25

# start_index = key_fp.find(value_fp[0][0].text)
# end_index = start_index+len(value_fp[0][0].text)

# key_fp[start_index:end_index]

# exact = value_fp[0][0].text
# prefix = key_fp[max(0,start_index-offset):start_index-1]
# postfix = key_fp[end_index+1:end_index+offset]
            
# [prefix, exact, postfix]      

# # make sure the index starts from 0, if the start_index-offset<0, set it 0
# # the total offset incuding prefix and posfix is 50
# 

# value_fp[0][0].attrs['ids'].split(',')[0]
# normalisation = 'http://purl.uniprot.org/uniprot/'+value_fp[0][0].attrs['ids'].split(',')[0]
# normalisation